# Mesh generation for tutorial 02

In [ ]:
import os
import meshio
import pygmsh
from dolfinx import MPI
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
pre_step_length = 4.
after_step_length = 14.
pre_step_height = 3.
after_step_height = 5.
lcar = 1. / 5.

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
p0 = geom.add_point([0.0, after_step_height - pre_step_height, 0.0], lcar)
p1 = geom.add_point([pre_step_length, after_step_height - pre_step_height, 0.0], lcar)
p2 = geom.add_point([pre_step_length, 0.0, 0.0], lcar)
p3 = geom.add_point([pre_step_length + after_step_length, 0.0, 0.0], lcar)
p4 = geom.add_point([pre_step_length + after_step_length, after_step_height, 0.0], lcar)
p5 = geom.add_point([0.0, after_step_height, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p2)
l2 = geom.add_line(p2, p3)
l3 = geom.add_line(p3, p4)
l4 = geom.add_line(p4, p5)
l5 = geom.add_line(p5, p0)
geom.add_physical([l5], label=1)
geom.add_physical([l0, l1, l2, l4], label=2)
line_loop = geom.add_line_loop([l0, l1, l2, l3, l4, l5])
domain = geom.add_plane_surface(line_loop)
geom.add_physical(domain, label=0)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Save temporary mesh (pygmsh format)

In [ ]:
meshio.write("backward_facing_step_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"triangle": pygmsh_mesh.cells_dict["triangle"]}
))

### Save temporary boundaries (pygmsh format)

In [ ]:
meshio.write("backward_facing_step_boundaries_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"line": pygmsh_mesh.cells_dict["line"]},
    cell_data={"boundaries": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]]}
))

### Read back in mesh (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "backward_facing_step_tmp.xdmf", "r") as infile:
    mesh = infile.read_mesh(name="Grid")
    mesh.name = "mesh"

In [ ]:
plot(mesh)

### Read back in boundaries (dolfinx format)

In [ ]:
mesh.create_connectivity_all()
with XDMFFile(MPI.comm_world, "backward_facing_step_boundaries_tmp.xdmf", "r") as infile:
    boundaries = infile.read_meshtags(mesh, name="Grid")
    boundaries.name = "boundaries"

### Save final mesh and boundaries (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "backward_facing_step.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(boundaries)

In [ ]:
os.remove("backward_facing_step_tmp.xdmf")
os.remove("backward_facing_step_tmp.h5")
os.remove("backward_facing_step_boundaries_tmp.xdmf")
os.remove("backward_facing_step_boundaries_tmp.h5")